In [7]:
import pygtrie
import re

from tqdm import tqdm

from litecoder.db import City

In [2]:
City.query.count()

54285

In [3]:
def keys_iter(row):
    
    states = (row.state_name, row.state_abbr)
    
    for state in states:
        yield '%s %s' % (row.name, state)
    
    if row.population and row.population > 500000:
        yield row.name

In [4]:
def make_key(text, lower=True):
    """Normalize text string -> index key.
    """
    text = text.strip()
    
    # Remove periods
    text = re.sub('\.', '', text)
    
    # Comma -> space
    text = re.sub(',', ' ', text)
    
    # 2+ whitespace -> space
    text = re.sub('\s{2,}', ' ', text)

    if lower:
        text = text.lower()

    return text

In [41]:
idx = pygtrie.StringTrie(separator=' ')

for c in tqdm(City.query):
    for key_raw in keys_iter(c):
        
        key = make_key(key_raw)
        
        if key in idx:
            idx[key] = (*idx[key], c.wof_id)
            
        else:
            idx[key] = (c.wof_id,)

54285it [00:04, 11996.96it/s]


In [46]:
%time idx.longest_prefix(make_key('hayfork ca'))

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 35.8 µs


('hayfork ca', (85927687,))